# NumPyro Sample 実行

## Libs

In [1]:
import numpyro
import numpyro.distributions as dist

import jax
import arviz as az

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
jax.devices()

[CpuDevice(id=0)]

## Model 定義

In [ ]:
def model(y = None, num_data = 0):

    # パラメーターの事前分布
    mu = numpyro.sample('mu', dist.HalfNormal(10))

    # 観測データ（Y）に基づく尤度の定義
    with numpyro.plate('data', num_data):
        # plateはfor文的な処理。グラフィカルモデル用語。
        
        numpyro.sample('obs', dist.Poisson(mu), obs = y)

## 推論

In [ ]:
nuts = numpyro.infer.NUTS(model)
mcmc = numpyro.infer.MCMC(nuts, num_warmup = 500, num_samples = 3000, num_chains = 4)
mcmc.run(jax.random.PRNGKey(42), y = y, num_data = len(y))
mcmc_samples = mcmc.get_samples()

## 分析準備

In [ ]:
# InferenceData オブジェクトへの変換（az のフォーマットに変換してあげるだけ）
idata = az.from_numpyro(mcmc)

## 収束チェック

In [ ]:
az.plot_trace(idata)

In [ ]:
# 基本的には R hat が 1.1未満ならOK
az.summary(idata)

In [ ]:
# 基本的には HDI をチェック（Highest Density Interval 最高密度区間 の略）実際にMCMCサンプリングした94%がここに入ってるよていう話
az.plot_posterior(idata)